In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL of the main page containing links to Lovecraft's stories
BASE_URL = "https://www.hplovecraft.com/writings/texts/"

# Function to get all story links from the main page
def get_story_links(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Store all links for each category (Fiction, Poetry, Essays, Letters)
    story_links = []
    for category in soup.select("h2"):
        # Get all links within the category
        for link in category.find_next("ul").find_all("a"):
            story_url = link['href']
            story_links.append(story_url)
    return story_links

# Function to scrape each story's title and text
def scrape_story(url):
    story_response = requests.get(url)
    story_soup = BeautifulSoup(story_response.text, 'html.parser')
    
    # Attempting multiple methods to capture the title
    title = story_soup.find('h1').text if story_soup.find('h1') else (
            story_soup.find('title').text if story_soup.find('title') else "Unknown Title")
    
    paragraphs = story_soup.find_all('p')
    story_text = "\n".join([para.text for para in paragraphs])
    
    return {"title": title, "url": url, "text": story_text}

# Main code to collect all stories and save to a CSV
def main():
    story_links = get_story_links(BASE_URL)
    stories = []
    
    for link in story_links:
        try:
            story = scrape_story(link)
            stories.append(story)
            print(f"Scraped: {story['title']}")
            time.sleep(1)  # Pause to avoid overloading the server
        except Exception as e:
            print(f"Failed to scrape {link}: {e}")
    
    # Save to CSV
    df = pd.DataFrame(stories)
    df.to_csv("lovecraft_stories.csv", index=False)
    print("Stories saved to lovecraft_stories.csv")

if __name__ == "__main__":
    main()
